set_subset and draw_system_graph from eps-net repository by Patrick Emonts 2023

In [8]:
import networkx as nx
import matplotlib.pyplot as plt
import random

In [39]:
import ast
def parse_tuple(string):
    try:
        s = ast.literal_eval(str(string))
        if type(s) == tuple:
            return s
        return
    except:
        return

def set_subset(dest, n):
    # Distribute subsets according to the sites (this makes drawing easier)
    nodes = []
    for node in dest.nodes():
        if node != 's' and node != 't':
            nodes.append(parse_tuple(node))
    
    subsetdict = {str((x,i)):{"subset":x} for (x,i) in nodes}
    nx.set_node_attributes(dest,subsetdict)
    dest.nodes['s']['subset'] = -1
    dest.nodes['t']['subset'] = n

def draw_system_graph(graph,ax=None):
    """Draw the connectivity graph for the nsite system.
    This graph is fully connected between the different sites.
    The number of nodes in each layer is given by the number of points in the respective eps-net on the site.

    Args:
        ax (plt.axis, optional): Matplotlib axis to use for drawing. Defaults to None.
    """
    f, axvec = plt.subplots(1,1)
    pos = nx.multipartite_layout(graph)
    labels = nx.get_edge_attributes(graph,'weight')
    new_labels = {}
    for k, v in labels.items():
        new_labels[k] = round(v, 2)

    pred, dist = nx.dijkstra_predecessor_and_distance(graph.reverse(copy=True), 't')
    #get edges in pred
    cedges = []
    for u in pred:
        for v in pred[u]:
            cedges.append((u, v))

    edge_color = []
    for e in graph.edges():
        if e in cedges:
            edge_color.append('red')
        else:
            edge_color.append('black')

    nx.draw_networkx(graph, pos, ax=ax,with_labels=True)
    nx.draw_networkx_edges(graph, pos, width=2.0, edge_color=edge_color)
    # nx.draw_networkx_edge_labels(graph, pos, ax=ax,edge_labels=new_labels)

In [53]:
l = [
    "ising_nsites_2_npoints_4.gml",
    "ising_nsites_3_npoints_4.gml",
    "ising_nsites_4_npoints_4.gml",
    "ising_nsites_5_npoints_4.gml",
    "ising_nsites_5_npoints_6.gml",
    "ising_nsites_5_npoints_10.gml",
    "ising_nsites_5_npoints_20.gml",
]

#g = nx.read_graphml("ising_nsites_2_npoints_4.gml")
#g = nx.read_graphml("ising_nsites_3_npoints_4.gml")
#g = nx.read_graphml("ising_nsites_4_npoints_4.gml")
#g = nx.read_graphml("ising_nsites_5_npoints_10.gml")
#g = nx.read_graphml("ising_nsites_5_npoints_20.gml")

for path in l:
    g = nx.read_graphml(path)

    for e in g.edges():
        if e[0] != 's' and e[1] != 't':
            g.edges[e]['weight'] += random.random() / 10**6
            
            # g.edges[e]['weight'] = int(random.random()*100)
            # g.edges[e]['weight'] = round(random.random(), 2)*100

    nx.write_graphml(g, "perturbated/p_" + path)

# set_subset(g, 10)
# draw_system_graph(g)
# plt.show()